In [9]:
# standard
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
from math import sqrt
import time

# reading data
import os
import json
from collections import defaultdict

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.fft import rfft, irfft, fftn, ifftn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

# visuals
import matplotlib.pyplot as plt
import seaborn as sns

# transformer Benchmarks
from Benchmarks.models.Informer import Model as Informer 
from Benchmarks.models.Autoformer import Model as Autoformer
from Benchmarks.models.Transformer import Model as VanillaTransformer

%store -r Kelmarsh_df Penmanshiel_df

# Args Dictionary

required for running all benchmark models

In [ ]:
hyperparameters = {}

# CRPS Loss Function

In [11]:
class CRPS(nn.Module):
    def __init__(self, custom_parameters):
        super(CRPS, self).__init__()
        self.custom_parameters = custom_parameters

    def forward(self, forecast, observations, weights):
        """
        Args:
        forecast (torch.Tensor): Forecasts from the model (ensemble) with shape [1, seq_len].
        observations (torch.Tensor): Observed values with shape [seq_len].
        weights (torch.Tensor): Corresponding weights for the CRPS scores, derived from sparse attention, with shape [1, seq_len, seq_len].

        Returns:
        float: Weighted mean of the CRPS for all forecasts.
        """
        forecast = forecast.squeeze(0)  # Adjusting forecast shape: [64]
        weights = weights.mean(dim=-1).squeeze(0)  # Assuming averaging is the method to obtain weights: [64]
    
        # Sorting the forecasts
        sorted_forecast, _ = torch.sort(forecast, dim=0)
        observations = observations.unsqueeze(0)  # [1, 64] for broadcasting

        # Cumulative sum of sorted forecasts
        cumsum_forecast = torch.cumsum(sorted_forecast, dim=0) / forecast.size(0)

        # Calculating CRPS
        indicator = (sorted_forecast > observations).float()
        differences = (cumsum_forecast - indicator) ** 2
        weighted_differences = differences * weights  # Apply weights to the differences
        crps = weighted_differences.mean()  # Taking mean across all weighted differences

        return crps  # Returning as a Tensor for the backward pass